In [4]:
pip install mido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.5 MB/s eta 0:00:00


In [24]:
#human readable
import mido

def read_midi_file(file_path):
    try:
        # Load the MIDI file
        midi = mido.MidiFile(file_path)

        # Display file metadata
        print(f"File: {file_path}")
        print(f"Type: {midi.type}")
        print(f"Number of tracks: {len(midi.tracks)}")
        print(f"Ticks per beat: {midi.ticks_per_beat}\n")

        # Iterate over the tracks and messages
        for i, track in enumerate(midi.tracks):
            print(f"Track {i}: {track.name}")
            for msg in track:
                print(msg)
            print("\n")

    except Exception as e:
        print(f"Error reading MIDI file: {e}")

# Example usage:
# Replace 'example.mid' with the path to your MIDI file
read_midi_file('senbonzakura.mid')


File: senbonzakura.mid
Type: 1
Number of tracks: 2
Ticks per beat: 384

Track 0: 
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=400000, time=0)
MetaMessage('end_of_track', time=136416)


Track 1: Electric Piano
MetaMessage('track_name', name='Electric Piano', time=1152)
program_change channel=0 program=0 time=0
note_on channel=0 note=62 velocity=50 time=0
note_off channel=0 note=62 velocity=0 time=192
note_on channel=0 note=62 velocity=50 time=192
note_on channel=0 note=60 velocity=50 time=288
note_off channel=0 note=62 velocity=0 time=0
note_on channel=0 note=62 velocity=50 time=96
note_off channel=0 note=60 velocity=0 time=0
note_on channel=0 note=65 velocity=50 time=192
note_off channel=0 note=62 velocity=0 time=0
note_off channel=0 note=65 velocity=0 time=96
note_on channel=0 note=65 velocity=50 time=96
note_off channel=0 note=65 velocity=0 time=96
note_on channel=0 note=67 veloci

In [25]:
#(pitch, start time, end time)
import mido

def extract_notes(file_path):
    try:
        midi = mido.MidiFile(file_path)
        notes = []
        current_time = 0
        active_notes = {}  # Tracks currently active notes {pitch: start_time}

        # Process each track
        for i, track in enumerate(midi.tracks):
            print(f"Processing Track {i}: {track.name}")
            current_time = 0  # Reset time for each track

            for msg in track:
                # Increment current time by delta
                current_time += mido.tick2second(msg.time, midi.ticks_per_beat, 500000)  # Default tempo: 500,000 microseconds per beat

                if msg.type == 'note_on' and msg.velocity > 0:
                    # Note On message
                    active_notes[msg.note] = current_time
                elif (msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0)):
                    # Note Off message
                    if msg.note in active_notes:
                        note_on_time = active_notes.pop(msg.note)
                        notes.append((msg.note, note_on_time, current_time))

        # Print extracted notes
        print("Extracted Notes (Pitch, Note On Time, Note Off Time):")
        for note in notes:
            print(note)

        return notes

    except Exception as e:
        print(f"Error processing MIDI file: {e}")
        return []

# Example usage:
# Replace 'example.mid' with the path to your MIDI file
notes_data = extract_notes('senbonzakura.mid')


Processing Track 0: 
Processing Track 1: Electric Piano
Extracted Notes (Pitch, Note On Time, Note Off Time):
(62, 1.5, 1.75)
(62, 2.0, 2.375)
(60, 2.375, 2.5)
(62, 2.5, 2.75)
(65, 2.75, 2.875)
(65, 3.0, 3.125)
(67, 3.25, 3.5)
(62, 3.5, 3.875)
(62, 4.0, 4.375)
(60, 4.375, 4.5)
(62, 4.5, 4.75)
(60, 4.75, 5.0)
(57, 5.0, 5.25)
(60, 5.25, 5.5)
(62, 5.5, 5.75)
(62, 6.0, 6.375)
(60, 6.375, 6.5)
(62, 6.5, 6.75)
(65, 6.75, 7.0)
(67, 7.0, 7.25)
(69, 7.25, 7.375)
(69, 7.5, 7.875)
(67, 8.0, 8.25)
(69, 8.25, 8.375)
(67, 8.375, 8.5)
(65, 8.5, 8.75)
(62, 9.0, 9.25)
(62, 9.5, 9.75)
(62, 10.0, 10.25)
(57, 10.375, 10.5)
(60, 10.5, 10.75)
(62, 10.75, 11.0)
(65, 11.0, 11.25)
(67, 11.25, 11.5)
(62, 11.5, 11.875)
(62, 12.0, 12.25)
(60, 12.25, 12.5)
(62, 12.5, 12.75)
(60, 12.75, 13.0)
(57, 13.0, 13.125)
(57, 13.25, 13.5)
(62, 13.5, 13.875)
(62, 14.0, 14.25)
(60, 14.25, 14.375)
(60, 14.5, 14.75)
(62, 14.75, 15.0)
(65, 15.0, 15.25)
(67, 15.25, 15.5)
(69, 15.5, 16.0)
(67, 16.0, 16.25)
(69, 16.25, 16.375)
(67, 

In [26]:
#array
import mido

def midi_to_pitch_array(file_path, interval=0.125):
    try:
        midi = mido.MidiFile(file_path)
        notes = []
        active_notes = {}  # Tracks active notes {pitch: start_time}
        current_time = 0

        # Calculate the maximum time length of the MIDI file
        max_time = sum(mido.tick2second(msg.time, midi.ticks_per_beat, 500000) for track in midi.tracks for msg in track)
        array_length = int(max_time / interval) + 1

        # Initialize the pitch array with 0
        pitch_array = [0] * array_length

        # Process each track
        for track in midi.tracks:
            current_time = 0  # Reset time for each track
            for msg in track:
                # Increment time by delta
                current_time += mido.tick2second(msg.time, midi.ticks_per_beat, 500000)

                # Handle Note On
                if msg.type == 'note_on' and msg.velocity > 0:
                    active_notes[msg.note] = current_time

                # Handle Note Off
                elif (msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0)):
                    if msg.note in active_notes:
                        note_on_time = active_notes.pop(msg.note)
                        note_off_time = current_time

                        # Find the start and end intervals
                        start_index = int(note_on_time / interval)
                        end_index = int(note_off_time / interval)

                        # Fill the pitch array for this note's duration
                        for i in range(start_index, end_index):
                            if pitch_array[i] == 0:
                                pitch_array[i] = [msg.note]
                            else:
                                pitch_array[i].append(msg.note)

        # Flatten intervals with multiple notes into a list of pitches
        pitch_array = [item if isinstance(item, list) else [item] if item != 0 else 0 for item in pitch_array]

        return pitch_array

    except Exception as e:
        print(f"Error processing MIDI file: {e}")
        return []

# Example usage
pitch_array = midi_to_pitch_array('senbonzakura.mid', interval=0.125)
print("Pitch Array:", pitch_array)


Pitch Array: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, [62], [62], 0, 0, [62], [62], [62], [60], [62], [62], [65], 0, [65], 0, [67], [67], [62], [62], [62], 0, [62], [62], [62], [60], [62], [62], [60], [60], [57], [57], [60], [60], [62], [62], 0, 0, [62], [62], [62], [60], [62], [62], [65], [65], [67], [67], [69], 0, [69], [69], [69], 0, [67], [67], [69], [67], [65], [65], 0, 0, [62], [62], 0, 0, [62], [62], 0, 0, [62], [62], 0, [57], [60], [60], [62], [62], [65], [65], [67], [67], [62], [62], [62], 0, [62], [62], [60], [60], [62], [62], [60], [60], [57], 0, [57], [57], [62], [62], [62], 0, [62], [62], [60], 0, [60], [60], [62], [62], [65], [65], [67], [67], [69], [69], [69], [69], [67], [67], [69], [67], [65], [65], [65], [65], [62], [62], [62], [62], [65], [65], [65], [65], [64], [64], [64], [64], [62], [62], [62], [62], [60], [60], 0, 0, [60], [60], [62], [60], [57], [57], [55], [55], [57], [57], [57], [57], 0, 0, 0, 0, [57], [57], [60], [60], [62], [62], 0, 0, [67], [67], [67], [67], [6

In [28]:
#hex string
import mido

def midi_to_pitch_array(file_path, interval=0.125, base_pitch=55):
    try:
        midi = mido.MidiFile(file_path)
        active_notes = {}  # Tracks active notes {pitch: start_time}
        current_time = 0

        # Calculate the maximum time length of the MIDI file
        max_time = sum(mido.tick2second(msg.time, midi.ticks_per_beat, 500000) for track in midi.tracks for msg in track)
        array_length = int(max_time / interval) + 1

        # Initialize the pitch array with 0
        pitch_array = [0] * array_length

        # Process each track
        for track in midi.tracks:
            current_time = 0  # Reset time for each track
            for msg in track:
                # Increment time by delta
                current_time += mido.tick2second(msg.time, midi.ticks_per_beat, 500000)

                # Handle Note On
                if msg.type == 'note_on' and msg.velocity > 0:
                    active_notes[msg.note] = current_time

                # Handle Note Off
                elif (msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0)):
                    if msg.note in active_notes:
                        note_on_time = active_notes.pop(msg.note)
                        note_off_time = current_time

                        # Find the start and end intervals
                        start_index = int(note_on_time / interval)
                        end_index = int(note_off_time / interval)

                        # Fill the pitch array for this note's duration
                        for i in range(start_index, end_index):
                            if pitch_array[i] == 0:
                                pitch_array[i] = [msg.note]
                            else:
                                pitch_array[i].append(msg.note)

        # Flatten intervals with multiple notes into a list of pitches
        pitch_array = [item if isinstance(item, list) else [item] if item != 0 else 0 for item in pitch_array]

        # Modify the pitch array and convert to hex string
        hex_string = ""
        for item in pitch_array:
            if item == 0:
                hex_string += "00"  # Add "00" for silence
            else:
                # Process each pitch in the item (list of pitches)
                hex_values = [format(pitch - base_pitch, '02x') for pitch in item]
                hex_string += "".join(hex_values)  # Combine hex values

        return hex_string

    except Exception as e:
        print(f"Error processing MIDI file: {e}")
        return ""

# Example usage
hex_result = midi_to_pitch_array('senbonzakura.mid', interval=0.125, base_pitch=54)
print("Hex String:", hex_result)


Hex String: 000000000000000000000000080800000808080608080b000b000d0d08080800080808060808060603030606080800000808080608080b0b0d0d0f000f0f0f000d0d0f0d0b0b0000080800000808000008080003060608080b0b0d0d080808000808060608080606030003030808080008080600060608080b0b0d0d0f0f0f0f0d0d0f0d0b0b0b0b080808080b0b0b0b0a0a0a0a08080808060600000606080603030101030303030000000003030606080800000d0d0d0d0a0a0a0a0b0b0b0b0a0a060608080808080808080b0b0b0b0a0a0a0a08080808060606000606080603030101030300000303060608000808000008080b0b0b0b0d0d0d0d0a0a0a0a0a0a0a0a0000000008080b0b0d0d000d0d0d0f000f0f0f0f00000f0f121214140d0d0b0b0f0f000008080b0b0d0d000d0d0d0f000f0f0f0f00000f0f10100f0f0d0d0b000b0b000008080b0b0d0d000d0d0d0f000f0f0f0f0f000f0f121214140d0d0b0b0f0f000008080b0b101010100f0f0f0f0d0d0d0d0b0b0b0b0d0d0f0f0a0a06060808000008080b0b0d0d000d0d0d0f000f0f0f0f00000f0f121214140d0d0b0b0f0f000008080b0b0d0d000d0d0d0f000f0f0f0f00000f0f10100f0f0d0d0b000b0b000008080b0b0d0d000d0d0d0f000f0f0f0f0f000f0f121214140d0d0b0b0f0f000008080b0b1010